# Data Preparation

## Packages

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import winsound
import datetime as dt

import math, collections
from scipy.stats import linregress

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope(y_axis):
    global rsi_window
    x_axis = []
    for i in range(rsi_window):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

## Read data

In [3]:
raw_df = pd.read_csv('data\M1_2019.csv')

#raw_df['tick'] = (raw_df['High'] - raw_df['Low'])/2
raw_df['tick'] = raw_df['Close']

df = raw_df[['TS', 'tick']]
print(f'{len(df)} records in df')
df.head()

372530 records in df


,TS,tick
0,2019-01-01 17:02,1.14598
1,2019-01-01 17:03,1.14607
2,2019-01-01 17:04,1.14606
3,2019-01-01 17:05,1.14621
4,2019-01-01 17:06,1.14665


## Parameters

In [4]:
min_bar = 2

pip_diff = 0.0001
diff_col = 'ssma'

rsi_window = 10
rs_max = 1e6

window_s_ma = 10
window_l_ma = 100

## Data manipulation

In [5]:
%%time
df = df.iloc[::min_bar]
print(f'{len(df)} records under {min_bar} min_bar')

df['diff'] = df['tick'].diff()

df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])

df['avg_gain'] = df['gain'].rolling(window=rsi_window).mean()
df['avg_loss'] = df['loss'].rolling(window=rsi_window).mean()

df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= rs_max, rs_max) 

df['rsi'] = 100 - (100 / (df['rs'] + 1))

df['ssma'] = df['tick'].rolling(window=window_s_ma).mean()
df['lsma'] = df['tick'].rolling(window=window_l_ma).mean()
df['sma_diff'] = df['ssma'] - df['lsma']

df['sema'] = df['tick'].ewm(span=window_s_ma).mean()
df['lema'] = df['tick'].ewm(span=window_l_ma).mean()
df['ema_diff'] = df['sema'] - df['lema']

df['slope'] = df['tick'].rolling(window=rsi_window).apply(get_slope)
df['sma_slope'] = df['ssma'].rolling(window=rsi_window).apply(get_slope)

df['target'] = 'same'

df['target'].loc[df[diff_col].shift(-1) - df[diff_col] > pip_diff] = 'increase'
df['target'].loc[df[diff_col].shift(-1) - df[diff_col] < -pip_diff] = 'decrease'

df = df.dropna()
df = df.reset_index(drop=True)

186265 records under 2 min_bar
Wall time: 4min 24s


## Write data to csv

In [6]:
del df['TS']
df.to_csv('data/tab_df.csv', index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

## Print Report

In [7]:
target_col = 'target'

print(f'Record count : {len(df)}')
print('--------------------------')
print(df[target_col].value_counts())
print('--------------------------')
print(df[target_col].value_counts(normalize=True))

df.head(10)

Record count : 186166
--------------------------
same        180369
decrease      2992
increase      2805
Name: target, dtype: int64
--------------------------
same        0.968861
decrease    0.016072
increase    0.015067
Name: target, dtype: float64


,tick,diff,gain,loss,avg_gain,avg_loss,rs,rsi,ssma,lsma,sma_diff,sema,lema,ema_diff,slope,sma_slope,target
0,1.14541,0.00009,0.00009,0.00000,0.000038,0.000060,0.633333,38.775510,1.145300,1.146044,-0.000744,1.145337,1.145868,-0.000531,-52.366769,-74.080770,same
1,1.14547,0.00006,0.00006,0.00000,0.000044,0.000048,0.916667,47.826087,1.145296,1.146039,-0.000743,1.145361,1.145859,-0.000498,45.855097,-73.563328,same
2,1.14541,-0.00006,0.00000,0.00006,0.000044,0.000045,0.977778,49.438202,1.145295,1.146032,-0.000737,1.145370,1.145849,-0.000479,68.029719,-71.767752,same
3,1.14533,-0.00008,0.00000,0.00008,0.000044,0.000038,1.157895,53.658537,1.145301,1.146019,-0.000718,1.145363,1.145837,-0.000474,68.222514,-68.346113,same
4,1.14526,-0.00007,0.00000,0.00007,0.000036,0.000045,0.800000,44.444444,1.145292,1.146011,-0.000719,1.145344,1.145824,-0.000480,69.007147,-62.391360,same
5,1.14526,0.00000,0.00000,0.00000,0.000036,0.000026,1.384615,58.064516,1.145302,1.146003,-0.000701,1.145329,1.145811,-0.000482,57.196955,-51.763105,same
6,1.14521,-0.00005,0.00000,0.00005,0.000036,0.000029,1.241379,55.384615,1.145309,1.145994,-0.000685,1.145307,1.145798,-0.000490,-1.735705,-34.405234,same
7,1.14526,0.00005,0.00005,0.00000,0.000041,0.000026,1.576923,61.194030,1.145324,1.145986,-0.000662,1.145299,1.145786,-0.000487,-58.380571,-11.309932,same
8,1.14531,0.00005,0.00005,0.00000,0.000026,0.000026,1.000000,50.000000,1.145324,1.145978,-0.000654,1.145301,1.145775,-0.000474,-60.863287,9.056416,same
9,1.14526,-0.00005,0.00000,0.00005,0.000025,0.000031,0.806452,44.642857,1.145318,1.145970,-0.000652,1.145293,1.145763,-0.000470,-65.255314,18.027714,same
